Use Python to fetch country debt profiles. Use a horizontal bar chart, the y-axis is the list of all countries in the world (for example, Japan, Korea, Philippines, China, and more), and the x-axis is the total debt in $.  Make a ranking label on the outside and the right part of the bar, followed by the number of debt$ label

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import matplotlib.pyplot as plt

# ----------------------------
# 1) Fetch HTML from debtclock.io
# ----------------------------

url = "https://debtclock.io/"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# ----------------------------
# 2) Extract debt values
#    (debtclock.io uses JS but values appear in meta tags)
# ----------------------------

# The page includes debt values in text such as:
# "United States 39,071,563,336,400"
# China 20,631,866,464,225 etc

pattern = re.compile(r'([A-Za-z\s]+)(\d{1,3}(?:,\d{3})+)')
matches = pattern.findall(soup.text)

# Parse found country names and numbers
data = []
seen = set()
for match in matches:
    name, num = match
    name = name.strip()
    # Ensure we only get country like names, no duplicates
    if len(name.split()) <= 4 and name not in seen:
        try:
            # Remove commas and convert to int
            value = int(num.replace(",", ""))
            data.append((name, value))
            seen.add(name)
        except:
            continue

# If no matches found, fail gracefully
if not data:
    raise Exception("Could not parse national debt values from debtclock.io")

# ----------------------------
# 3) Prepare data for plotting
# ----------------------------

# Sort descending by debt
data_sorted = sorted(data, key=lambda x: x[1], reverse=True)

countries = [item[0] for item in data_sorted]
debts = [item[1] for item in data_sorted]
ranks = list(range(1, len(countries)+1))

# ----------------------------
# 4) Plot horizontal bar chart
# ----------------------------

plt.figure(figsize=(14, max(6, len(countries)/3)))
bars = plt.barh(countries, debts, color="navy")
plt.xlabel("National Debt (USD)")
plt.title("Real‑Time Estimates of Country National Debt (from DebtClock.io)")

# Add ranking and value labels
for bar, rank, debt in zip(bars, ranks, debts):
    width = bar.get_width()
    # Human readable
    if debt >= 1e12:
        val_label = f"{debt/1e12:.2f}T"
    elif debt >= 1e9:
        val_label = f"{debt/1e9:.2f}B"
    else:
        val_label = f"{debt:,}"

    plt.text(
        width * 1.01,
        bar.get_y() + bar.get_height() / 2,
        f"#{rank} {val_label}",
        va="center",
        fontsize=8
    )

plt.tight_layout()
plt.show()
